## 이슈트래커 키워드 추출하기

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
project_id = os.environ.get('GOOGLE_CLOUD_PROJECT_ID')

In [2]:
# 스파크 설정
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, udf

spark = SparkSession.builder\
  .appName('issue-tracker')\
  .config('spark.jars', '../spark-3.3-bigquery-0.32.0.jar')\
  .getOrCreate()

23/08/12 12:18:04 WARN Utils: Your hostname, HwangonJang-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.29 instead (on interface en0)
23/08/12 12:18:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/08/12 12:18:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# timestamp udf 설정
from datetime import datetime, timedelta

@udf('string')
def datetime_to_string(dt):
  dt -= timedelta(hours=9)
  return dt.strftime('%Y-%m-%d %H:%M:%S')

@udf('string')
def datetime_to_string_minus(min):
  dt = datetime.now()
  dt -= timedelta(minutes=min)
  return dt.strftime('%Y-%m-%d %H:%M:%S')

spark.udf.register("datetime_to_string", datetime_to_string)
spark.udf.register("datetime_to_string_minus", datetime_to_string_minus)

<function __main__.datetime_to_string_minus(min)>

In [4]:
# 데이터 조회
df = spark.read \
  .format('bigquery') \
  .load(f'{project_id}.news.news')

df.createOrReplaceTempView("news")

In [5]:
# 시간 truncate by 10 minutes
def truncate_seconds(dt, interval_minutes):
    truncated_minutes = int(dt.minute / interval_minutes) * interval_minutes
    return dt.replace(minute=truncated_minutes, second=0, microsecond=0)

interval_minutes = 10  # 잘라낼 분 단위

In [6]:
# 하루 동안의 뉴스 수집
query = """
SELECT
    id,
    title,
    to_timestamp(datetime_to_string(article_written_at)) as article_written_at,
    to_timestamp(datetime_to_string(scraped_at)) as scraped_at,
    category
FROM
    news
WHERE
    to_timestamp(datetime_to_string(article_written_at)) >= to_timestamp(datetime_to_string_minus(60 * 24))
    AND to_timestamp(datetime_to_string(article_written_at)) < to_timestamp(datetime_to_string_minus(0))
"""

# 스파크 SQL query 실행
issue_df = spark.sql(query)

In [7]:
# lazy execution - 실제로 값이 필요할 때 계산
issue_df.show()

+----------+---------------------------------+-------------------+-------------------+--------+
|        id|                            title| article_written_at|         scraped_at|category|
+----------+---------------------------------+-------------------+-------------------+--------+
|0005171937|  월가의 터줏대감 세 사람, 한 ...|2023-08-12 11:38:01|2023-08-12 11:40:00|    경제|
|0014127827| 로마 콜로세움서 현피?…머스크 ...|2023-08-12 11:33:37|2023-08-12 11:40:00|    경제|
|0005550108| ‘중국에 투자하지마’…바이든 엄...|2023-08-12 11:31:01|2023-08-12 11:40:00|    경제|
|0004879252| 카트 끌다 사망했는데 '과태료 ...|2023-08-12 11:11:01|2023-08-12 11:20:00|    경제|
|0000068654|   "당근 3개로 피부톤을 바꿔?"...|2023-08-12 11:01:01|2023-08-12 11:10:00|    과학|
|0005550106|  "음주도 아닌데 이상한데?"…뇌...|2023-08-12 11:15:07|2023-08-12 11:20:00|    사회|
|0002182724|  “복막염 증상” 긴급수술 받은 ...|2023-08-12 11:33:01|2023-08-12 11:40:00|    사회|
|0003384409|   “농락과 학대 수준”…‘재력가 ...|2023-08-12 11:27:01|2023-08-12 11:30:00|    사회|
|0002819030|새벽에 양손에 흉기 들고…파출소...|2023-08-12 11:21

In [8]:
issue_df.createOrReplaceTempView("issue_single_day")

# 1시간 동안의 정치 카테고리 뉴스 수집
query = """
SELECT
    id,
    title,
    article_written_at,
    scraped_at,
    category
FROM
    issue_single_day
WHERE
    article_written_at >= to_timestamp(datetime_to_string_minus(60))
    AND article_written_at < to_timestamp(datetime_to_string_minus(0))
    AND category == '정치'
"""

# 스파크 SQL query 실행
issue_hour_df = spark.sql(query)

In [9]:
issue_hour_df.show()

+----------+--------------------------------+-------------------+-------------------+--------+
|        id|                           title| article_written_at|         scraped_at|category|
+----------+--------------------------------+-------------------+-------------------+--------+
|0003781298|이재명 ‘안면인식장애’ 진술에 ...|2023-08-12 11:52:01|2023-08-12 12:00:00|    정치|
|0003384412| 김현숙 ‘잼버리’ 책임론 주목…...|2023-08-12 11:43:01|2023-08-12 11:50:00|    정치|
|0004879254|     'D.P.' 정주행한 이재명…'...|2023-08-12 11:41:01|2023-08-12 11:50:00|    정치|
+----------+--------------------------------+-------------------+-------------------+--------+



In [15]:
# 데이터 전처리
import pandas as pd

pd_issue_df = issue_df.select('title', 'category').toPandas()
pd_issue_hour_df = issue_hour_df.select('title', 'category').toPandas()

issues = pd_issue_df.copy()
for _ in range(2):
    issues = pd.concat([issues, pd_issue_hour_df], ignore_index=True)

In [16]:
issues.head(5)

,title,category
0,"월가의 터줏대감 세 사람, 한 명이 감쪽같이 사라졌다...이유는 놀랍게도 [흥부전]",경제
1,"로마 콜로세움서 현피?…머스크 ""伊와 논의"" 저커버그 ""합의 아냐""(종합)",경제
2,‘중국에 투자하지마’…바이든 엄포에 난감해진 자본시장,경제
3,카트 끌다 사망했는데 '과태료 내면 그만'…불통의 코스트코 [박동휘의 컨슈머 리포트],경제
4,"""당근 3개로 피부톤을 바꿔?""...난리난 '태닝법'의 진짜 효과는?",과학


In [17]:
from konlpy.tag import Okt
import re

# Okt 객체 생성
okt = Okt()

def remove_bracket_text(title):
    return re.sub(r'\[.*?\]', '', title)

# '정치' 카테고리에 해당하는 모든 제목을 합친 후 명사 추출 작업 수행
category_politics = issues[issues['category'] == '정치']
category_politics['title'] = category_politics['title'].apply(remove_bracket_text)  # 대괄호 안 신문사 이름 삭제

all_titles = ' '.join(category_politics['title'])
tokens_const = okt.nouns(all_titles)

const_cnt = {}
max_words = 20
for word in tokens_const:
    const_cnt[word] = const_cnt.get(word, 0) + 1
sorted_w = sorted(const_cnt.items(), key=lambda kv: kv[1])
result = sorted_w[-max_words:]
result.reverse()

print(result)

[('수사', 15), ('잼버리', 14), ('책임', 12), ('이재명', 12), ('사건', 10), ('채', 10), ('해병대', 10), ('신혼부부', 9), ('청약', 9), ('대출', 9), ('론', 8), ('정부', 8), ('단장', 8), ('국민', 7), ('수근', 7), ('상병', 7), ('민주당', 7), ('힘', 6), ('김현숙', 6), ('기준', 6)]


/var/folders/6c/_n3059wn75j1nxgw4xdk_yr40000gn/T/ipykernel_64693/2156817998.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_politics['title'] = category_politics['title'].apply(remove_bracket_text)  # 대괄호 안 신문사 이름 삭제
